### We are assuming that the state vector are in the following order:
$$ x = [E_{qp}\quad E_{dp}\quad \delta\quad \omega\quad V_F\quad V_A\quad V_E]^T $$

In [1]:
from rushisland5 import *

In [2]:
import numpy as np
import numdifftools as nd
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline 
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from scipy import optimize
np.set_printoptions(precision=25)

In [3]:
# Pe = Eq*Iq + Ed*Id = Eq*(Vd-Ed)/Xqp + Ed*(Eq-Vq)/Xdp = (Eq*Vd - Ed*Vq)/Xdp
def f_w1(x):
#     Pm = x[14]
    Pm = 6.8954233610562881
#     Pm = 7.0
    Eqp1 = x[0]
    Edp1 = x[1]
    Vd1 = np.real(f_Vdq(x)[0])
    Vq1 = np.imag(f_Vdq(x)[0])
    Pe = (Eqp1*Vd1 - Edp1*Vq1)/Xdp
    return 1 / (2 * H) * (Pm - Pe)


def f_w2(x):
#     Pm = x[15]
    Pm = 7.5312971941631437
#     Pm = 7.49
    Eqp2 = x[7]
    Edp2 = x[8]
    Vd2 = np.real(f_Vdq(x)[1])
    Vq2 = np.imag(f_Vdq(x)[1])
    Pe = (Eqp2*Vd2 - Edp2*Vq2)/Xdp
    return 1 / (2 * H) * (Pm - Pe)


def f_VA1(x):
#     Vref = x[16]
    Vref = 1.0586038003890010
    Eqp1 = x[0]
    Edp1 = x[1]
    Vf1 = x[4]
    Va1 = x[5]
    Ve1 = x[6]
    Id1 = f_Id1(x)
    XadIfd = Eqp1 + (Xd - Xdp) * Id1
    Vfe = KD1 * XadIfd + KE1 * Ve1 + Aex1 * np.exp(Bex1 * Ve1)
    yf = KF1 / TF1 * (Vfe - Vf1)
    Vsum = Vref - np.absolute(f_Vdq(x)[0]) - yf
    return (Vsum - Va1) / TA1


def f_VA2(x):
#     Vref = x[17]
    Vref = 1.0659163034132286
    Eqp2 = x[7]
    Edp2 = x[8]
    Vf2 = x[11]
    Va2 = x[12]
    Ve2 = x[13]
    Id2 = f_Id2(x)
    XadIfd = Eqp2 + (Xd - Xdp) * Id2
    Vfe = KD2 * XadIfd + KE2 * Ve2 + Aex2 * np.exp(Bex2 * Ve2)
    yf = KF2 / TF2 * (Vfe - Vf2)
    Vsum = Vref - np.absolute(f_Vdq(x)[1]) - yf
    return (Vsum - Va2) / TA2


def sys_fun(x):
    fun = [f_Eqp1, f_Edp1, f_delta1, f_w1, f_VF1, f_VA1, f_VE1, f_Eqp2, f_Edp2, f_delta2, f_w2, f_VF2, f_VA2, f_VE2]
    
#     J = np.array([nd.Jacobian(f)(x).ravel() for f in fun])
#     J = J[:,:14]
#     lam, v = np.linalg.eig(J)
#     #lam = lam[abs(lam.imag).argsort()][::-1]
#     print(lam)
#     res = np.append(np.array([f(x).ravel() for f in fun]).ravel(), [lam[4].real,lam[5].real,lam[8].real,lam[9].real])
#     return res

    return np.array([f(x).ravel() for f in fun]).ravel()

all_fun = [f_Eqp1,f_Edp1,f_delta1,f_w1,f_VF1,f_VA1,f_VE1,f_Eqp2,f_Edp2,f_delta2,f_w2,f_VF2,f_VA2,f_VE2]

In [4]:
def Quassian(x0):
    """
    This function calculates the 4th order derivative of a function f: R^n -> R^n from its 3rd order derivative
    return: 5-D matrix, because of f:R^n -> R^n
    """
    Quassian = np.zeros((x0.shape[0],x0.shape[0],x0.shape[0],x0.shape[0],x0.shape[0]))
    for i in range(x0.shape[0]):
        h = 1e-4
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        Quassian[:,i] = (Trissian(xm2) - 8*Trissian(xm1) + 8*Trissian(xp1) - Trissian(xp2))/(12*h)
    return Quassian

def Quissian(x0):
    """
    This function calculates the 5th order derivative of a function f: R^n -> R^n from its 4th order derivative
    return: 6-D matrix, because of f:R^n -> R^n
    """
    Quissian = np.zeros((x0.shape[0],x0.shape[0],x0.shape[0],x0.shape[0],x0.shape[0],x0.shape[0]))
    for i in range(x0.shape[0]):
        h = 1e-4
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        Quissian[:,i] = (Quassian(xm2) - 8*Quassian(xm1) + 8*Quassian(xp1) - Quassian(xp2))/(12*h)
    return Quissian

In [5]:
sys_fun(x)

array([ 2.4462400002247348e-13,  8.9385122874669238e-12,
       -1.9884958138562446e-26, -1.1799436589489680e-12,
       -1.9938961914717304e-12, -2.0336221132991977e-11,
        1.7626925560202485e-12,  1.9030014170685610e-13,
        1.5969425519637771e-11, -3.6531015914904231e-26,
        5.1407361776014735e-13, -9.3614005436393199e-13,
       -7.4997300036905301e-14,  1.2001795568768359e-12])

In [ ]:
# sol = optimize.root(sys_fun, x, method='hybr')
# print(sol.fun)
# print(sol.message)
# print(sol.success)

In [ ]:
# x = sol.x

In [6]:
n_c = 4
n_s = 10
n = x.shape[0]

#J= np.array([nd.Jacobian(f)(x).ravel() for f in all_fun])
J = Jacobian(x)
lam, v = np.linalg.eig(J)
idx = abs(lam.imag).argsort()
lam = lam[idx]
v = v[:,idx]

In [7]:
""" Weijun's Formula """
idx = np.where(abs(lam.real) < 1e-8)[0]

Q = np.c_[v[:,idx[0]].imag, v[:,idx[0]].real, v[:,idx[2]].imag, v[:,idx[2]].real]
i=0
while i < len(lam):
    if i in idx:
        i += 1
    else:
        if lam[i].imag == 0:
            Q=np.c_[Q,v[:,i]]
            i += 1
        else:
            Q=np.c_[Q,v[:,i].imag,v[:,i].real]
            i += 2
            
Q = Q/(abs(np.linalg.det(Q))**(1/n))
Q = np.real(Q)
P = np.linalg.inv(Q)
J_cs = np.linalg.multi_dot([P,J,Q])
#display(pd.DataFrame(J_cs).applymap(lambda x: '{:,.8f}'.format(x)))

In [8]:
pd.DataFrame(J_cs)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000000,-2.001308,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
1,2.001308,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000
2,0.000000,-0.000000,-0.000000,-5.195755,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000
3,-0.000000,-0.000000,5.195755,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000
4,0.000000,0.000000,-0.000000,-0.000000,-6.575645,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000
5,0.000000,0.000000,-0.000000,-0.000000,0.000000,-4.721139,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000
6,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,-2.831054,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000
7,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-1.067009,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000
8,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-1.040445,-0.921900,-0.000000,-0.000000,-0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.921900,-1.040445,-0.000000,-0.000000,0.000000,0.000000


In [ ]:
a = np.arange(4).reshape(2,2)
b = np.arange(4).reshape(2,2)
print(b.T.dot(a).dot(b))
print(a.reshape(1,-1).dot(np.kron(b,b)))
a = np.einsum('ij,jq->iq',a, b)
a = np.einsum('ij,iq->qj',a, b)
print(a)

In [ ]:
a = np.arange(27).reshape(3,3,3)
b = np.arange(9).reshape(3,3)

print(a.reshape(1,-1).dot(np.kron(np.kron(b,b),b)))
a = np.einsum('ijk,kq->ijq',a, b)
a = np.einsum('ijk,jq->iqk',a, b)
a = np.einsum('ijk,iq->qjk',a, b)
print(a)

In [9]:
Z2 = Hessian(x)/2
for i in range(n):
    Z2[i] = np.einsum('ij,jq->iq',Z2[i], Q)
    Z2[i] = np.einsum('ij,iq->qj',Z2[i], Q)
Z2 = Z2.reshape(n,-1)
# Z2_Gain = []
# for i in itertools.product(range(n),repeat=2):
#     val = 1
#     for j in range(n):
#         val *= math.factorial(i.count(j))
#     Z2_Gain.append(1/val)
# Z2 = Z2 * np.array(Z2_Gain)
# fullidx = list(itertools.product(range(n),repeat=2))
# partidx = list(itertools.combinations_with_replacement(range(n), 2))
# idx = [fullidx.index(x) for x in partidx if x in fullidx]
# W2 = np.linalg.inv(Q).dot(Z2[:,idx])

W2 = np.linalg.inv(Q).dot(Z2)

In [10]:
Z3 = Trissian(x)/6
for i in range(n):
    Z3[i] = np.einsum('ijk,kq->ijq',Z3[i], Q)
    Z3[i] = np.einsum('ijk,jq->iqk',Z3[i], Q)
    Z3[i] = np.einsum('ijk,iq->qjk',Z3[i], Q)
Z3 = Z3.reshape(n,-1)

W3 = np.linalg.inv(Q).dot(Z3)

In [11]:
Z4 = Quassian(x)/24
for i in range(n):
    Z4[i] = np.einsum('ijkl,lq->ijkq',Z4[i], Q)
    Z4[i] = np.einsum('ijkl,kq->ijql',Z4[i], Q)
    Z4[i] = np.einsum('ijkl,jq->iqkl',Z4[i], Q)
    Z4[i] = np.einsum('ijkl,iq->qjkl',Z4[i], Q)
Z4 = Z4.reshape(n,-1)

W4 = np.linalg.inv(Q).dot(Z4)

In [12]:
Z5 = Quissian(x)/120
for i in range(n):
    Z5[i] = np.einsum('ijklh,hq->ijklq',Z5[i], Q)
    Z5[i] = np.einsum('ijklh,lq->ijkqh',Z5[i], Q)
    Z5[i] = np.einsum('ijklh,kq->ijqlh',Z5[i], Q)
    Z5[i] = np.einsum('ijklh,jq->iqklh',Z5[i], Q)
    Z5[i] = np.einsum('ijklh,iq->qjklh',Z5[i], Q)
Z5 = Z5.reshape(n,-1)

W5 = np.linalg.inv(Q).dot(Z5)

In [13]:
J_c = J_cs[0:n_c,0:n_c]
J_s = J_cs[n_c:,n_c:]

In [14]:
xx_idx = [i*n + j for i in range(n_c)   for j in range(n_c)]
xy_idx = [i*n + j for i in range(n_c)   for j in range(n_c,n)]
yy_idx = [i*n + j for i in range(n_c,n) for j in range(n_c,n)]

f2_xx = np.array([w[xx_idx]   for w in W2[0:n_c]])
f2_xy = np.array([w[xy_idx]*2 for w in W2[0:n_c]])
f2_yy = np.array([w[yy_idx]   for w in W2[0:n_c]])

g2_xx = np.array([w[xx_idx]   for w in W2[n_c:]])
g2_xy = np.array([w[xy_idx]*2 for w in W2[n_c:]])

In [15]:
xxx_idx = [i*n**2 + j*n + k for i in range(n_c)   for j in range(n_c)   for k in range(n_c)]
xxy_idx = [i*n**2 + j*n + k for i in range(n_c)   for j in range(n_c)   for k in range(n_c,n)]
xyy_idx = [i*n**2 + j*n + k for i in range(n_c)   for j in range(n_c,n) for k in range(n_c,n)]

f3_xxx = np.array([w[xxx_idx]   for w in W3[0:n_c]])
f3_xxy = np.array([w[xxy_idx]*3 for w in W3[0:n_c]])
f3_xyy = np.array([w[xyy_idx]*3 for w in W3[0:n_c]])

g3_xxx = np.array([w[xxx_idx]   for w in W3[n_c:]])

In [16]:
xxxx_idx = [i*n**3 + j*n**2 + k*n + h  for i in range(n_c) for j in range(n_c) for k in range(n_c) for h in range(n_c)]
xxxy_idx = [i*n**3 + j*n**2 + k*n + h  for i in range(n_c) for j in range(n_c) for k in range(n_c) for h in range(n_c,n)]

f4_xxxx = np.array([w[xxxx_idx]     for w in W4[0:n_c]])
f4_xxxy = np.array([w[xxxy_idx]*4   for w in W4[0:n_c]])

In [17]:
import itertools
xxxxx_idx = [i*n**4 + j*n**3 + k*n**2 + h*n + l  for i,j,k,h,l in itertools.product(range(n_c),repeat=5)]

f5_xxxxx = np.array([w[xxxxx_idx]     for w in W5[0:n_c]])

In [18]:
""" Solve for H_c2, H_c3 in y = H_c2*Bx2 + H_c3*Bx3 for center manifold calculation"""

I4 = np.eye(4)
I16 = np.eye(16)

"""C2*Bx2 = DBx2 * Jc * x """
C2 = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
H_c2 = sp.linalg.solve_sylvester(J_s, -C2, -g2_xx)

"""C3*Bx3 = DBx3 * Jc * x"""
C3 = np.kron(I16, J_c) + np.array([np.kron(I16, row) for row in J_c]).reshape(64,-1) + \
     np.kron(I4, np.array([np.kron(I4, row) for row in J_c]).reshape(16,-1))  
H_c3 = sp.linalg.solve_sylvester(J_s, -C3,\
                                 H_c2.dot(np.kron(I4, f2_xx) + \
                                          np.array([np.kron(I4,row) for row in f2_xx]).reshape(16,-1)) \
                                 - g2_xy.dot(np.kron(I4, H_c2)) - g3_xxx
                                )

In [19]:
""" Plug y = H_c2*Bx2 + H_c3*Bx3 into center manifold and reformulize center manifold """

f2_cm = f2_xx
""" 3rd order: f3_cm (cm ~ center manifold) """
f3_cm = np.linalg.multi_dot([f2_xy, np.kron(np.eye(n_c), H_c2)]) + f3_xxx

""" 4th order: f4_cm"""
f4_cm = f2_xy.dot(np.kron(I4, H_c3)) + f2_yy.dot(np.kron(H_c2,H_c2)) + f3_xxy.dot(np.kron(I16, H_c2)) + f4_xxxx

""" 5th order: f5_cm"""
I64 = np.eye(64)
f5_cm = f2_yy.dot(np.kron(H_c2,H_c3) + np.kron(H_c3,H_c2)) + f3_xxy.dot(np.kron(I16,H_c3)) \
        + f3_xyy.dot(np.kron(I4, np.kron(H_c2,H_c2))) + f4_xxxy.dot(np.kron(I64,H_c2)) + f5_xxxxx

""" Now center manifold up to 5th order is:

  xdot = Jc*x + f2_xx*Bx2 + f3_cm*Bx3 + f4_cm*Bx4 + f5_cm*Bx5 + O(6) 
  
"""

' Now center manifold up to 5th order is:\n\n  xdot = Jc*x + f2_xx*Bx2 + f3_cm*Bx3 + f4_cm*Bx4 + f5_cm*Bx5 + O(6) \n  \n'

In [21]:
""" Now we can use normal form to simplify the center manifold"""
""" Solve H2n in normal form transformation x = y + H2n * By2"""
H_2n = sp.linalg.solve_sylvester(J_c, -C2, -f2_xx)
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,-C2) + f2_xx
#pd.DataFrame(R_2n)

""" Now calculate the 3,4,5 order terms in center manifold after the transformation x = y + H2n * By2"""

Ny3 = f2_xx.dot(np.kron(I4,H_2n) + np.kron(H_2n,I4)) + f3_cm
#pd.DataFrame(Ny3)
Ny4 = f2_xx.dot(np.kron(H_2n,H_2n)) + f3_cm.dot(np.kron(np.kron(I4,H_2n) + np.kron(H_2n,I4),I4) + np.kron(I16,H_2n)) + f4_cm \
      - H_2n.dot(np.kron(I4,Ny3) + np.array([np.kron(I4,row) for row in Ny3]).reshape(16,-1))
Ny5 = f3_cm.dot(np.kron(np.kron(H_2n,H_2n),I4) + np.kron(np.kron(I4,H_2n) + np.kron(H_2n,I4), H_2n)) + \
      f4_cm.dot(np.kron(np.kron(np.kron(I4,H_2n) + np.kron(H_2n,I4),I4) + np.kron(I16,H_2n), I4) + np.kron(I64,H_2n)) + f5_cm \
      - H_2n.dot(np.kron(I4,Ny4) + np.array([np.kron(I4,row) for row in Ny4]).reshape(16,-1)) # This term is like C2

""" Now center manifold up to 5th order is:

  ydot = Jc*y + Ny3*By3 + Ny4*By4 + Ny5*By5 + O(6) 
  
"""

' Now center manifold up to 5th order is:\n\n  ydot = Jc*y + Ny3*By3 + Ny4*By4 + Ny5*By5 + O(6) \n  \n'

In [33]:
""" Solve H3n in normal form transformation y = z + H3n*Bz3"""
Lambda3 = np.kron(J_c,np.eye(n_c**3)) + np.kron(np.eye(n_c),-C3.T)
lam, v = np.linalg.eig(Lambda3)
zero_rows = np.where(abs(lam) < 1e-6)[0]
nzero_rows = np.where(abs(lam) > 1e-6)[0]

h3 = np.zeros(4*64,dtype=np.complex64)
h3[nzero_rows] = np.linalg.inv(v).dot(-Ny3.reshape((1,-1)).T)[nzero_rows].ravel() / lam[nzero_rows]
H_3n = v.dot(h3).reshape(4,-1)

""" Now calculate the 3,4,5 order terms in center manifold after the transformation y = z + H3n*Bz3"""
# Nz3 = np.zeros((4*64,1),dtype=np.complex64)
# Nz3[zero_rows] = np.linalg.inv(v).dot(Ny3.reshape((1,-1)).T)[zero_rows]
# Nz3 = v.dot(Nz3).reshape(4,-1)
Nz3 = J_c.dot(H_3n) - H_3n.dot(C3) + Ny3

Nz4 = Ny4
Nz5 = Ny3.dot(np.kron(I16,H_3n) + np.kron(np.kron(I4,H_3n) + np.kron(H_3n,I4) ,I4)) + Ny5 - \
      H_3n.dot(np.kron(np.eye(n_c**2), Nz3) + np.array([np.kron(np.eye(n_c**2), row) for row in Nz3]).reshape(-1,n_c**5)+\
               np.kron(np.eye(n_c),np.array([np.kron(np.eye(n_c), row) for row in Nz3]).reshape(-1,n_c**4))
              )

""" Now center manifold up to 5th order is:

  zdot = Jc*z + Nz3*Bz3 + Nz4*Bz4 + Nz5*Bz5 + O(6) 
  
"""

' Now center manifold up to 5th order is:\n\n  zdot = Jc*z + Nz3*Bz3 + Nz4*Bz4 + Nz5*Bz5 + O(6) \n  \n'

In [44]:
import itertools
R_3n = np.zeros((Nz3.shape[0],n_c*(n_c+1)*(n_c+2)//6),dtype=np.complex64)
idx = list(itertools.combinations_with_replacement(range(n_c), 3))
for i,j,k in itertools.product(range(n_c),repeat=3):
    R_3n[:,idx.index(tuple(sorted((i,j,k))))] += Nz3[:,i*n_c**2+j*n_c**1+k]

# print(list(itertools.combinations_with_replacement(range(1,n_c+1), 3)))
print(pd.DataFrame(R_3n.real))
a1_r = R_3n.real.reshape(1,-1)[:,[0,4,21,30]]
print(a1_r)
a2_r = R_3n.real.reshape(1,-1)[:,[7,9,33,35]]
print(a2_r)
b1_r = R_3n.real.reshape(1,-1)[:,[56,58,77,79]]
print(b1_r)
b2_r = R_3n.real.reshape(1,-1)[:,[42,51,63,72]]
print(b2_r)

         0         1         2         3         4         5         6   \
0  0.037949 -0.095597 -0.000000  0.000000  0.037949  0.000001 -0.000000   
1  0.095597  0.037949  0.000000 -0.000000  0.095597 -0.000001  0.000000   
2 -0.000000  0.000000 -0.097499  0.433359  0.000000  0.000000 -0.000002   
3 -0.000000 -0.000000 -0.433359 -0.097501 -0.000000 -0.000003  0.000000   

         7         8        9         10        11        12        13  \
0  0.665499  0.000001 0.665498 -0.095597  0.000000 -0.000000 -1.384299   
1  1.384299  0.000000 1.384299  0.037950  0.000000  0.000000  0.665499   
2 -0.000000  0.000000 0.000000  0.000000 -0.097501  0.433359  0.000000   
3  0.000000 -0.000000 0.000000  0.000000 -0.433359 -0.097499  0.000000   

        14        15        16        17        18        19  
0 0.000000 -1.384299  0.000000 -0.000000 -0.000000  0.000000  
1 0.000000  0.665498  0.000000  0.000000 -0.000000  0.000000  
2 0.000000  0.000000 -0.379324  0.809608 -0.379324  0.809608  
3

In [45]:
""" Solve H4n in normal form transformation z = w + H4n*Bw4"""
Nw3 = Nz3
"""C4*Bx4 = DBx4 * Jc * x"""
C4 = np.kron(I64, J_c) + \
     np.array([np.kron(I64, row) for row in J_c]).reshape(256,-1) + \
     np.kron(I4,  np.array([np.kron(I16, row) for row in J_c]).reshape(64,-1)) + \
     np.kron(I16, np.array([np.kron(I4,  row) for row in J_c]).reshape(16,-1))

C4_r = np.zeros((C4.shape[0],35),dtype=np.float)
idx = list(itertools.combinations_with_replacement(range(n_c), 4))
for i,j,k,h in itertools.product(range(n_c),repeat=4):
    C4_r[:,idx.index(tuple(sorted((i,j,k,h))))] += C4[: ,i*n_c**3 + j*n_c**2 + k*n_c + h]
C4_r = C4_r[[i*n_c**3 + j*n_c**2 + k*n_c + h for i,j,k,h in itertools.combinations_with_replacement(range(n_c), 4)], :]
#pd.DataFrame(C4_r)

Lambda4 = np.kron(J_c,np.eye(4*5*6*7//24)) + np.kron(I4,-C4_r.T)
lam, _ = np.linalg.eig(Lambda4)
print(np.where(abs(lam) < 1e-6)[0])

""" Here, knowing the all the eigenvalues of lambda4 are not on imaginary axis (Lambda4 is full rank), we know that all 4th 
    order terms would be eliminated; Also we don't need to calc H_4n, since it won't effect 5th order further """

# H_4n = sp.linalg.solve_sylvester(J_c, -C4, -Nz4)
# R_4n = np.dot(J_c,H_4n) + np.dot(H_4n,-C4) + Nz4

Nw5 = Nz5
""" Now center manifold up to 5th order is:

  wdot = Jc*w + Nw3*Bw3 + Nw5*Bw5 + O(6) 
  
"""


[]


' Now center manifold up to 5th order is:\n\n  wdot = Jc*w + Nw3*Bw3 + Nw5*Bw5 + O(6) \n  \n'

In [62]:
""" Solve H5n in normal form transformation w = v + H5n*Bv5"""
Nv3 = Nw3
"""C5*Bx5 = DBx5 * Jc * x"""
I256 = np.eye(256)
C5 = np.kron(I256, J_c) + \
     np.array([np.kron(I256, row) for row in J_c]).reshape(1024,-1) + \
     np.kron(I4,  np.array([np.kron(I64, row) for row in J_c]).reshape(256,-1)) + \
     np.kron(I16, np.array([np.kron(I16, row) for row in J_c]).reshape(64,-1)) + \
     np.kron(I64, np.array([np.kron(I4,  row) for row in J_c]).reshape(16,-1))

C5_r = np.zeros((C5.shape[0],56),dtype=np.float)
Nv5 = np.zeros((Nw5.shape[0],56),dtype=np.complex)
idx = list(itertools.combinations_with_replacement(range(n_c), 5))
for i,j,k,h,l in itertools.product(range(n_c),repeat=5):
    C5_r[:,idx.index(tuple(sorted((i,j,k,h,l))))] +=  C5[: ,i*n_c**4 + j*n_c**3 + k*n_c**2 + h*n_c + l]
    Nv5[:,idx.index(tuple(sorted((i,j,k,h,l))))] += Nw5[: ,i*n_c**4 + j*n_c**3 + k*n_c**2 + h*n_c + l]
C5_r = C5_r[[i*n_c**4 + j*n_c**3 + k*n_c**2 + h*n_c + l for i,j,k,h,l in itertools.combinations_with_replacement(range(n_c), 5)], :]
# pd.DataFrame(C5_r)


Lambda5 = np.kron(J_c,np.eye(56)) + np.kron(np.eye(n_c),-C5_r.T)
lam, v = np.linalg.eig(Lambda5)
zero_rows = np.where(abs(lam) < 1e-6)[0]
nzero_rows = np.where(abs(lam) > 1e-6)[0]

# h5 = np.zeros(4*1024,dtype=np.complex64)
# h5[nzero_rows] = np.linalg.inv(v).dot(-Nw5.reshape((1,-1)).T)[nzero_rows].ravel() / lam[nzero_rows]
# H_5n = v.dot(h5).reshape(4,-1)

R_5n = np.zeros((4*56,1),dtype=np.complex64)
R_5n[zero_rows] = np.linalg.inv(v).dot(Nv5.reshape((1,-1)).T)[zero_rows]
R_5n = v.dot(R_5n).reshape(4,-1)

In [74]:
pd.DataFrame(R_5n.real)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
0,-0.040133,-0.068816,0.000000,-0.000000,-0.080266,0.000000,0.000000,0.504311,-0.000000,0.504311,-0.137633,0.000000,-0.000000,-3.163504,0.000000,-3.163504,-0.000000,0.000000,0.000000,0.000000,-0.040133,-0.000000,0.000000,0.504311,0.000000,0.504311,0.000000,0.000000,0.000000,0.000000,0.357519,-0.000000,0.715039,0.000000,0.357519,-0.068816,0.000000,-0.000000,-3.163504,0.000000,-3.163504,-0.000000,0.000000,-0.000000,0.000000,-1.617657,0.000000,-3.235314,0.000000,-1.617657,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000
1,0.068816,-0.040133,-0.000000,-0.000000,0.137633,-0.000000,-0.000000,3.163504,-0.000000,3.163504,-0.080266,0.000000,-0.000000,0.504311,0.000000,0.504311,-0.000000,-0.000000,-0.000000,-0.000000,0.068816,0.000000,0.000000,3.163504,-0.000000,3.163504,-0.000000,-0.000000,-0.000000,-0.000000,1.617657,-0.000000,3.235314,-0.000000,1.617657,-0.040133,-0.000000,0.000000,0.504311,0.000000,0.504311,-0.000000,-0.000000,-0.000000,-0.000000,0.357519,-0.000000,0.715039,-0.000000,0.357519,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2,-0.000000,-0.000000,-0.250774,0.272224,-0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.501549,0.544448,-0.000000,0.000000,0.000000,-0.805225,0.916523,-0.805225,0.916523,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.250774,0.272224,0.000000,-0.000000,-0.000000,-0.805225,0.916523,-0.805225,0.916523,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.209980,-0.087704,-0.419959,-0.175409,-0.209980,-0.087704
3,0.000000,-0.000000,-0.272224,-0.250774,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.544448,-0.501549,-0.000000,-0.000000,-0.000000,-0.916523,-0.805225,-0.916523,-0.805225,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.272224,-0.250774,-0.000000,0.000000,-0.000000,-0.916523,-0.805225,-0.916523,-0.805225,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.087704,-0.209980,0.175409,-0.419959,0.087704,-0.209980


In [86]:
for i in enumerate(itertools.combinations_with_replacement(range(4), 5)):
    if abs(R_5n[2,i[0]].real) > 1e-6:
        print(i[0], i[1], R_5n[2,i[0]].real) 

2 (0, 0, 0, 0, 2) -0.250774351868448
3 (0, 0, 0, 0, 3) 0.27222394163528396
11 (0, 0, 1, 1, 2) -0.5015487037368939
12 (0, 0, 1, 1, 3) 0.5444478832705727
16 (0, 0, 2, 2, 2) -0.8052253722743263
17 (0, 0, 2, 2, 3) 0.916523339463474
18 (0, 0, 2, 3, 3) -0.8052253722743276
19 (0, 0, 3, 3, 3) 0.9165233394634691
36 (1, 1, 1, 1, 2) -0.25077435186844454
37 (1, 1, 1, 1, 3) 0.2722239416352872
41 (1, 1, 2, 2, 2) -0.8052253722743279
42 (1, 1, 2, 2, 3) 0.9165233394634749
43 (1, 1, 2, 3, 3) -0.8052253722743257
44 (1, 1, 3, 3, 3) 0.9165233394634686
50 (2, 2, 2, 2, 2) -0.20997973543890724
51 (2, 2, 2, 2, 3) -0.08770446437175172
52 (2, 2, 2, 3, 3) -0.4199594708778083
53 (2, 2, 3, 3, 3) -0.175408928743505
54 (2, 3, 3, 3, 3) -0.20997973543890164
55 (3, 3, 3, 3, 3) -0.08770446437175383


$$ \dot{r_1} = r_1 (c_1^R r_1^4+c_2^R r_1^2 r_2^2+c_3^R r_2^4) $$ 
$$ \dot{r_2} = r_2 (d_1^R r_1^4+d_2^R r_1^2 r_2^2+d_3^R r_2^4) $$
$$ \dot{x_1} = (c_1^R x_1-c_1^I x_2 )(x_1^4+x_2^4 )+(c_2^R x_1-c_2^I x_2 )(x_1^2+x_2^2 )(x_3^2+x_4^2 )+(c_3^R x_1-c_3^I x_2 )(x_3^4+x_4^4 ) $$
$$ \dot{x_2} = (c_1^R x_2+c_1^I x_1 )(x_1^4+x_2^4 )+(c_2^R x_2+c_2^I x_1 )(x_1^2+x_2^2 )(x_3^2+x_4^2 )+(c_3^R x_2+c_3^I x_1 )(x_3^4+x_4^4 ) $$
$$ \dot{x_3} = (d_1^R x_3-d_1^I x_4 )(x_1^4+x_2^4 )+(d_2^R x_3-d_2^I x_4 )(x_1^2+x_2^2 )(x_3^2+x_4^2 )+(d_3^R x_3-d_3^I x_4 )(x_3^4+x_4^4 ) $$
$$ \dot{x_4} = (d_1^R x_4+d_1^I x_3 )(x_1^4+x_2^4 )+(d_2^R x_4+d_2^I x_3 )(x_1^2+x_2^2 )(x_3^2+x_4^2 )+(d_3^R x_4+d_3^I x_3 )(x_3^4+x_4^4 ) $$

In [90]:
c1_r = R_5n.real.reshape(1,-1)[:,[0,20,57,91]];print(c1_r)
c2_r = R_5n.real.reshape(1,-1)[:,[7,9,23,25,13+56,15+56,38+56,40+56]];print(c2_r)
c3_r = R_5n.real.reshape(1,-1)[:,[30,34,45+56,49+56]];print(c3_r)

d1_r = R_5n.real.reshape(1,-1)[:,[2+112,36+112,3+168,37+168]];print(d1_r)
d2_r = R_5n.real.reshape(1,-1)[:,[16+112,18+112,41+112,43+112,17+168,19+168,42+168,44+168]];print(d2_r)
d3_r = R_5n.real.reshape(1,-1)[:,[50+112,54+112,51+168,55+168]];print(d3_r)

[[-0.04013294858374454  -0.04013294858374092  -0.04013294858374657
  -0.040132948583744985]]
[[0.5043106643033497 0.5043106643033493 0.5043106643033612
  0.5043106643033566 0.5043106643033506 0.5043106643033526
  0.5043106643033441 0.5043106643033439]]
[[0.35751947294342734 0.35751947294342534 0.3575194729434256
  0.35751947294342257]]
[[-0.250774351868448   -0.25077435186844454 -0.25077435186844516
  -0.25077435186844677]]
[[-0.8052253722743263 -0.8052253722743276 -0.8052253722743279
  -0.8052253722743257 -0.8052253722743261 -0.8052253722743231
  -0.8052253722743234 -0.8052253722743237]]
[[-0.20997973543890724 -0.20997973543890164 -0.20997973543890758
  -0.20997973543890436]]


$$ \dot{z}=i\omega_1z+a_1z^2\bar{z}+a_2zw\bar{w}+\ldots $$ 
$$ \dot{w}=i\omega_2w+b_2z\bar{z}w+b_1w^2\bar{w}+\ldots $$


$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

$$ \dot{r_1}=\mu_1 r_1+a_1^R r_1^3+a_2^R r_1 r_2^2+\ldots $$ 
$$ \dot{r_2}=\mu_2 r_2+b_2^R r_1^2 r_2+b_1^R r_2^3+\ldots $$ 

In [ ]:
# array([[ 0.037949371410692066],
#        [ 0.0955972510391369  ],
#        [ 0.6654987536483867  ],
#        [ 1.3842986917945828  ],
#        [-0.37932413441618607 ],
#        [-0.8096079834388115  ],
#        [-0.09749996275140119 ],
#        [-0.4333591005915055  ]])

In [ ]:
""" Old """
# _, v = np.linalg.eig(J.T)
# Q1 = np.block([
#     [np.stack((v[:,4].real,v[:,4].imag,v[:,8].real,v[:,8].imag))],  # Here we know the 4-th and 8-th eigenvalues are what we interested
#     [np.eye(10),np.zeros((10,4))]
# ])
# J1 = np.dot(np.dot(Q1,J),np.linalg.inv(Q1))
# Q2 = np.block([
#     [np.eye(4), np.zeros((4,10))],
#     [sp.linalg.solve_sylvester(J1[4:,4:], -J1[0:4,0:4], J1[4:,0:4]), np.eye(10)] #AX+XB=Q
# ]) 
# Q = np.dot(Q2,Q1)

# J_cs = np.linalg.multi_dot([Q,J,np.linalg.inv(Q)])
# del Q1
# del Q2
# del J1
# display(pd.DataFrame(J_cs))

In [ ]:
""" Old method for Z2, W2 """
# Z2 = np.zeros((n,n*(n+1)//2))
# hes = Hessian(x)
# for i in range(n):
#     hes[i][np.triu_indices(n,1)] *= 2  # double each element above the main diagonal
#     Z2[i] = hes[i][np.triu_indices(n)] # Keep upper triangular part
# Z2 = Z2/2 #divide all elements by 2, which corresponds to *2 in above line
# W2 = np.linalg.multi_dot([np.linalg.inv(Q), Z2, T2_mat(n), np.kron(Q,Q), S2_mat(n)])

In [ ]:
""" Old method for Z3, W3"""
# Z3 = np.zeros((n,n*(n+1)*(n+2)//6))
# Z3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
# tri = Trissian(x)
# for i in range(n):
#     #t = Trissian(all_fun[i], x)
#     #t = Trissian(x)[i]
#     t = tri[i]
#     Z3[i] = [t[j] for j in Z3_idx]
    
# Z3_Gain = []
# for i in Z3_idx:
#     val = 1
#     for j in range(n):
#         val *= math.factorial(i.count(j))
#     Z3_Gain.append(1/val)
# Z3_Gain = np.diag(Z3_Gain)
# Z3 = Z3.dot(Z3_Gain)

# W3 = np.linalg.multi_dot([np.linalg.inv(Q), Z3, T3_mat(n), np.kron(np.eye(n),T2_mat(n)), 
#                       np.kron(np.kron(Q,Q), Q), 
#                       np.kron(np.eye(n),S2_mat(n)), S3_mat(n)])

In [ ]:
""" 2nd to Old method for Z2, W2"""
# import itertools
# Z2 = Hessian(x).reshape(n,-1)/2
# W2 = Z2.dot(np.kron(Q,Q))

# Z2_Gain = []
# for i in itertools.product(range(n),repeat=2):
#     val = 1
#     for j in range(n):
#         val *= math.factorial(i.count(j))
#     Z2_Gain.append(2/val)
# W2 = W2.dot(np.diag(Z2_Gain))

# fullidx = list(itertools.product(range(n),repeat=2))
# partidx = list(itertools.combinations_with_replacement(range(n), 2))
# idx = [fullidx.index(x) for x in partidx]
# W2 = np.linalg.inv(Q).dot(W2[:,idx])

In [ ]:
""" 2nd to Old method for Z3, W3"""
# Z3 = Trissian(x).reshape(n,-1)/6
# W3 = Z3.dot(np.kron(np.kron(Q,Q),Q))

# Z3_Gain = []
# for i in itertools.product(range(n),repeat=3):
#     val = 1
#     for j in range(n):
#         val *= math.factorial(i.count(j))
#     Z3_Gain.append(6/val)
# W3 = W3.dot(np.diag(Z3_Gain))

# fullidx = list(itertools.product(range(n),repeat=3))
# partidx = list(itertools.combinations_with_replacement(range(n), 3))
# idx = [fullidx.index(x) for x in partidx if x in fullidx]
# W3 = np.linalg.inv(Q).dot(W3[:,idx])

In [ ]:
# def Jacobian(f_test, x0):
#     """
#     This function calculates the 1st order derivative of a function f: R^n -> R
#     input: 
#         f_test is the function
#         x0 where the 1st order want to be calcuated
#     return: 1-D row vector
#     """
#     Jac = np.zeros(x0.shape[0])
#     for i in range(x0.shape[0]):
#         h = 1e-6
#         xp1 = np.array(x0, copy=True) 
#         xp1[i] += h
#         #print(xp1)
#         xp2 = np.array(x0, copy=True) 
#         xp2[i] += 2*h
#         #print(xp2)
#         xm1 = np.array(x0, copy=True) 
#         xm1[i] -= h
#         #print(xm1)
#         xm2 = np.array(x0, copy=True) 
#         xm2[i] -= 2*h
#         #print(xm2)
#         Jac[i] = (-f_test(xp2) + 8*f_test(xp1)- 8*f_test(xm1) + f_test(xm2))/(12*h)
#     return Jac

# def Hessian(f_test, x0):
#     """
#     This function calculates the 2nd order derivative of a function f: R^n -> R
#     input: 
#         f_test is the function
#         x0 where the 2nd order want to be calcuated
#     return: 2-D matrix
#     """
#     Hes = np.zeros((x0.shape[0],x0.shape[0]))
#     for i in range(x0.shape[0]):
#         h = 1e-4
#         xp1 = np.array(x0, copy=True) 
#         xp1[i] += h
#         #print(xp1)
#         xp2 = np.array(x0, copy=True) 
#         xp2[i] += 2*h
#         #print(xp2)
#         xm1 = np.array(x0, copy=True) 
#         xm1[i] -= h
#         #print(xm1)
#         xm2 = np.array(x0, copy=True) 
#         xm2[i] -= 2*h
#         #print(xm2)
#         Hes[i] = (-Jacobian(f_test, xp2) + 8*Jacobian(f_test, xp1)- 8*Jacobian(f_test, xm1) + Jacobian(f_test, xm2))/(12*h)
#     return Hes

# def Trissian(f_test, x0):
#     """
#     This function calculates the 3rd order derivative of a function f: R^n -> R
#     input: 
#         f_test is the function
#         x0 where the 3rd order want to be calcuated
#     return: 3-D matrix
#     """
#     Trissian = np.zeros((x0.shape[0],x0.shape[0],x0.shape[0]))
#     for i in range(x0.shape[0]):
#         h = 1e-4
#         xp1 = np.array(x0, copy=True) 
#         xp1[i] += h
#         #print(xp1)
#         xp2 = np.array(x0, copy=True) 
#         xp2[i] += 2*h
#         #print(xp2)
#         xm1 = np.array(x0, copy=True) 
#         xm1[i] -= h
#         #print(xm1)
#         xm2 = np.array(x0, copy=True) 
#         xm2[i] -= 2*h
#         #print(xm2)
#         #Trissian[i] = (-nd.Hessian(f_test)(xp2) + 8*nd.Hessian(f_test)(xp1)- 8*nd.Hessian(f_test)(xm1) + nd.Hessian(f_test)(xm2))/(12*h)
#         Trissian[i] = (-Hessian(f_test, xp2) + 8*Hessian(f_test, xp1)- 8*Hessian(f_test, xm1) + Hessian(f_test, xm2))/(12*h)
#     return Trissian

# Notations

$$ \dot{z} = f(z),\quad f(z_0)=0 $$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  \Delta z^T H \Delta z + H.O.T(z)$$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  Z_2*B_z^2 + H.O.T(z)$$
Let $w = Q\Delta z$, and ignore the $\Delta$ sign then
$$Q^{-1}\dot{w} = JQ^{-1} w + \frac{1}{2!} (Q^{-1}w)^T H  (Q^{-1}w) + H.O.T(Q^{-1}w)$$
$$\dot{w} = QJQ^{-1} w + Q \frac{1}{2!} w^T (Q^{-T}H Q^{-1})w + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!} w^T H_Q w + Q * H.O.T(Q^{-1}w)$$
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!}  \begin{bmatrix} w^T H_Q[0] w \\  w^T H_Q[1] w \\ ...  \end{bmatrix} + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
where $W_2$ $W_3$ are constant matrices and $B_w^2$ and $B_w^3$ are column vectors deriving from $w \otimes w$ and $w \otimes w \otimes w$

And we assume that $$w = [u \quad v]^T$$ where $u$ is center variable, $v$ is stable variable

Now assume $B^2_{u}$ is $u\otimes u$ (note that this is a long column vector, so we do not need the $vec$ sign) but ij terms only appear once: e.g. if $u = [u_1 u_2 u_3]^T$, then $B^2_{u}=[u_1^2, u_2u_1, u_3u_1, u_2^2, u_3u_2, u_3^2]$

So we also need two transformation matrices $T$ and $S$ which doing transformations between $B_{uu}$ and $u\otimes u$ like: $$B^2_{u}=T_2 (u\otimes u)$$  
$$u\otimes u=S_2 B^2_{u}$$ 

similarly we will have  
$$B^3_{u}=T_3 (u\otimes B^2_{u})$$  
$$(u\otimes B^2_{u})=S_3 B^3_{u}$$ 

### Now we calculate Jacobian matrix $J$ and  the tranformation matrix $Q$

### Now we can 2nd and 3rd order matrices $W_2$ and $W_3$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
Since originally we are calculating hessian matrices of $f(z)$, then we need to make transformation from $z$ to $w$, thus
$$ \frac{1}{2} Q Z_2 B_z^2 = \frac{1}{2} Q Z_2[T_2 (z \otimes z)] = \frac{1}{2} Q Z_2 T_2 (Q^{-1} w \otimes Q^{-1} w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})(w\otimes w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 B_w^2 = W_2 B_w^2$$
Thus
$$\frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 = W_2 $$

For 3rd order
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (z \otimes B_z^2) = \frac{1}{3!} Q Z_3 T_3 (z \otimes T_2(z \otimes z))$$
where

\begin{align}
&  z \otimes T_2(z \otimes z) \\
& = (I\otimes T_2)(z \otimes z \otimes z) \\
& = (I\otimes T_2)(Q^{-1}w \otimes Q^{-1}w \otimes Q^{-1}w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes w \otimes w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes S_2 B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 )(w \otimes B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3
\end{align}

Thus
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3 = W_3 B_w^3$$


### Now we are able to calculate center manifold up to 2nd order terms, which we denote it by:
$$v = h_{c2}(u) = \color{red}{H_{c2}}B_u^2$$
where $\color{red}{H_{c2}} \in \mathbb{R}^{n_s \times n_c(n_c+1)/2}$ is the matrix we want to find; the subscript $c$ means center manifold (to distinguish it from normal form later), and 2 means 2nd order

Then according to equation:
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
if we express the term $W_2 B_w^2$ in form of $u$ and $v$, we will have
$$ \dot{u} = J_c u + f_2(u,v) = J_c u + \color{red}{U_u^2} B_u^2 + \color{red}{U_{uv}} B_{uv} + U_v^2 B_v^2 + ... $$
$$ \dot{v} = J_s v + g_2(u,v) = J_s v + \color{red}{V_u^2} B_u^2 + V_{uv} B_{uv} + V_v^2 B_v^2 + ... $$

Since we can find $\color{red}{H_{c2}}$ by solving the sylvester equation: $\color{red}{J_s H_{c2} + H_{c2} C_2 = -V_u^2}$ so we need to calculate the matrix of $V_u^2$

where $C_2$ comes from
$$ (DB_u^2)J_c u = T_2 D(u \otimes u) J_c u = T_2 \bar{J_c}(u \otimes u) = T_2 \bar{J_c} S_2 B_u^2 = -C_2 B_u^2$$
and
$$\bar{J_c} = (I_n \otimes J_c) + \begin{pmatrix} I_n \otimes J_1\\I_n \otimes J_2 \\ ... \\I_n \otimes J_n \end{pmatrix}$$ where each $J_i$ is the i-th row of matrix $J_c$

Now we construct matrix <font color = 'red'>$V_u^2$, $\bar{J_c}$ $C_2$ $H_{c2}$ </font>

### Now we can plug $v = {H_{c2}}B_u^2$ into equation of $\dot{u}$, then we can get up to the 3rd order terms
Since we have 
$$ \dot{u} = J_c u + f_2(u,v) + f_3(u,v) + ... $$
by plugging 
$$v = h_{c2}(u) = H_{c2} B_u^2$$
into the above equation, for third order terms of $u$, we only need:
$$ \dot{u} = J_c u + f_2(u,H_{c2} B_u^2) + f_3(u,0) + ... $$
Let
$$f_2(u,v) = U_u^{2} B_u^2 + U_{uv} B_{uv} + U_v^{2} B_v^2$$
and 
$$f_3(u,0) = U_u^{3} B_u^3$$
Then we can see that $B_{uv}$ is in 3rd order of $u$. $B_v^2$ is in 4th order of $u$, which is ignored here.
So the center manifold up to 3rd order of $u$ could be expressed as:
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} B_{uv} + U_u^{3} B_u^3 + O(u^4) $$
Since 
$$B_{uv} = u \otimes v = u \otimes H_{c2} B_u^2 = (I_{n_c} \otimes H_{c2})(u \otimes B_u^2) = (I_{n_c} \otimes H_{c2})S_3 B_u^3$$
Thus
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} (I_{n_c} \otimes H_{c2})S_3 B_u^3 + U_u^{3} B_u^3 + O(u^4) = J_c u + \color{red}{U_u^{2}} B_u^2 + [\color{red}{U_{uv}} (I_{n_c} \otimes H_{c2})\color{red}{S_3} + \color{red}{U_u^{3}}] B_u^3 + O(u^4)$$

So here in order to get the know the equation of $\dot{u}$ up to 3rd order, we need to know the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$, $U_u^{3}$</font>.

Here in order to know $U_u^{3}$, which comes from $f_3(u,0)$, we need to calculate matrix $W_3$ in equation $\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$

### Firstly we calculate $U_u^2$ and $U_{uv}$. Both of them comes from $W_2$

### Now we are ready to calculate matrix $U_u^3$. 
But in order to calculate it, we need to calculate the 3rd order derivative matrix $W_3$, then formulize $U_u^3$

Now we have known the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$  and $U_u^{3}$ </font>

Then we can calculate 
$$U_{uc}^3 = U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}$$ 
in the center manifold:
$$\dot{u} =  J_c u + U_u^{2} B_u^2 + [U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}] B_u^3 + O(u^4) = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$

### <font color='red'>Now we've got the flow $\dot{u}$ on the center manifold up to 3rd order, the next step is to calculate the normal form</font>

<font color='red'>$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
至此，我们只需要研究这个方程，我们已经知道$J_c$, $U_u^{2}$, $U_{uc}$,那么接下来就是要算normalform
</font>
#### First, we calculate the normal form up to the 2nd order:
$$u = y + \color{red}{H_{2n}} B_y^2$$
where we want to find matrix $\color{red}{H_{2n}}\in \mathbb{R}^{n_c \times n_c(n_c+1)/2}$

Then by plugging it into the above equation, we will get:
$$(I + H_{2n} DB_y^2) \dot{y} = J_c (y + H_{2n} B_y^2) + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
<font color='red'>$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since by some derivation, we know:
$$B_u^2 = B_y^2 + T_2\bar{H}(I\otimes S_2)S_3 B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ ($T_2=T_2(n_c)$) and $h_1$ ... $h_n$ are rows of matrix $H$.
In the above equations, if we assume $u\in \mathbb{R}^{n_c}$, then $H_{2n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)}{2}}$, $H \in \mathbb{R}^{n_c\times n_c^2}$, $\bar{H} \in \mathbb{R}^{n_c^2 \times n_c^3}$, $S_2=S_2(n_c)$, $S_3=S_3(n_c)$

Then
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
**Then up to 2nd order terms on the right hand side is**:
$$\dot{y} = J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 - H_{2n} DB_y^2 J_c y + O(y^3)$$
If we denote $- DB_y^2 J_c y = \color{red}{C_2} B_y^2$ as we did in calculation of center manifold, then:
$$\dot{y} = J_c y + [J_c H_{2n} + U_u^2 + H_{2n} C_2] B_y^2 + O(y^3)$$
Due to resonance terms, $J_c H_{2n} + H_{2n} C_2 + U_u^2 $ might not be totally eliminated, so we let
<font color='red'>$$J_c H_{2n} + H_{2n} C_2 = - U_u^2 + R_{2n} $$</font>
where matrix $R_{2n}$ represents resonance terms in normal form.

Now the problem is trying to solve for $H_{2n}$ which makes $R_{2n}$ as simple as possible, where $J_c$, $U_{u}^2$ and $C_2$ are known

Since $J_c \in \mathbb{R}^{n_c\times n_c}$ , $H_{2n} \in \mathbb{R}^{n_c\times \frac{n_c(n_c+1)}{2}}$  and $C_2 \in \mathbb{R}^{\frac{n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)}{2}}$
First we could transform the equation into:
$$\underbrace{(I_{\frac{n_c(n_c+1)}{2}}\otimes J_c + C_2^T \otimes I_{n_c})}_\text{$\Lambda_2$} \cdot \text{vec} (H_{2n}) = -\text{vec}(U_u^2) + \text{vec}(R_{2n})$$


Since we know that for Hopf Bifurcation, the 2nd order terms in normal form will all be eliminated, so $\Lambda$ should be a full rank matrix and the resonance term $R_{2n}$ will not exist, which is also verified below. Then we could solve for $H_{2n}$ by directly solving the Sylvester equation $J_c H_{2n} + H_{2n} C_2 = - U_u^2$

### Next, we calculate the normal form up to the 3rd order terms
From previous equation:
$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since if we only consider 3rd order terms, $U_{uc}^3 B_u^3$ can be expressed as:
$$ U_{uc}^3 B_u^3 = U_{uc}^3 B_y^3$$
So 
$$\dot{y} = [I - \underbrace{H_{2n} DB_y^2}_\text{O(y)} + \underbrace{(H_{2n} DB_y^2)(H_{2n} DB_y^2)}_\text{O(y^2)}+O(y^3)] [J_c y + \underbrace{(J_c H_{2n} + U_u^2) B_y^2}_\text{O(y^2)} + \underbrace{[U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3}_\text{O(y^3)} + O(u^4)]$$
Thus the 3rd order terms on the right hand side is:

\begin{align}
& [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 (J_c H_{2n} + U_u^2) B_y^2 + (H_{2n} DB_y^2)(H_{2n} DB_y^2)J_c y \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 - H_{2n} DB_y^2 J_c y] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 + H_{2n} C_2 B_y^2] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} \underbrace{DB_y^2 R_{2n} B_y^2}_\text{O(y^3)} 
\end{align}

where the sizes of matrices are:
\begin{align}
& H_{2n}, U_{u2} --  n_c \times \frac{n_c(n_c+1)}{2}  \\
& T_2(n_c) -- \frac{n_c(n_c+1)}{2} \times n_c^2  \\
& \bar{H}, \bar{R} -- n_c^2 \times n_c^3  \\
& I\otimes S_2(n_c) -- (n_c \times n_c) \otimes (n_c^2 \times \frac{n_c(n_c+1)}{2})  \\
& S_3(n_c) -- \frac{n_c^2 n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
& U_{uc}^3 -- n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
\end{align}

So we only need to calculate 
$$ DB_y^2 R_{2n} B_y^2 = T_2 \bar{R} (I \otimes S_2) S_3 B_y^3$$
where $$\bar{R} = (I_n\otimes R) + \begin{pmatrix} I_n\otimes r_1 \\ I_n\otimes r_2 \\ \vdots \\ I_n\otimes r_n \end{pmatrix} $$
where $R = R_{2n}T_2$ and $r_i$ are rows of $R$

Thus the 3rd order terms are:
$$N_y^3 B_y^3 = [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3 - H_{2n} T_2 \bar{R} (I \otimes S_2) S_3 ]B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ and $h_1$ ... $h_n$ are rows of matrix $H$

Now the normal form is:
$$\dot{y} = J_c y + R_{2n}B_y^2 + N_y^3 B_y^3 + O(y^4)$$

Now we want to simply $N_y^3$ as many as possible, so we consider another 3rd order nonlinear transformation 
$$ y = z + H_{3n}B_z^3$$

Here if we assume $y,z \in \mathbb{R}^{n_c}$ then $H_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$

Similarly, we will have 
$$\dot{z} = J_c z + R_{2n}B_z^2 + R_{3n} B_z^3 + O(z^4)$$
where
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
where 
$$C_3 B_z^3 = -DB_z^3 J_c z$$ and $$C_3 = -T_3(I_n\otimes T_2)\bar{J}(I_n\otimes S_2)S_3$$
where
$$\bar{J} = (I_{n^2} \otimes J_c) + I_n \otimes \begin{pmatrix} I_n\otimes jc_1 \\I_n\otimes jc_2 \\ \vdots \\ I_n\otimes jc_n \\ \end{pmatrix} + \begin{pmatrix} I_{n^2}\otimes jc_1 \\I_{n^2}\otimes jc_2 \\ \vdots \\ I_{n^2}\otimes jc_n \\ \end{pmatrix}$$

$jc_i$ are rows of matrix $J_c$

So here we have $R_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, $C_3 \in \mathbb{R}^{\frac{n_c(n_c+1)(n_c+2)}{3!} \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, 

So in order to calculate $R_{3n}$, first we need to calculate $N_y^3$.
Let's calcuate $\bar{H}$ and $\bar{R}$ first

Then we tranform the equation:
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
into:
$$(I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}) * \text{vec}(H_{3n}) = \text{vec}(R_{3n}) - \text{vec}(N_y^3)$$
Let
$$\Lambda_3 = I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}$$
<font color='red'>
Here we are assuming the $vec$ operation is taking the column vectors of the matrix. However, it is more easily for us to interprete the results if we assume that $vec$ is taking the row vectors of the matrix. But here the $\Lambda_3$ would be changing to
    $$ \Lambda_3 = J_c \otimes I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} + I_{n_c} \otimes C_3^T$$
</font>

Now Since we know that the column vector $-vec(N_y^3)$ is in the column space of matrix $\Lambda_3$ but vector $vec(R_3^n)$ is not. This means vector $vec(R_3^n)$ is in the complement space of the column space of matrix $\Lambda_3$. Since we have already know what the normal form with 3rd order resonance terms looks like (shown below), so we could easiliy know the basis of the space where vector $vec(R_3^n)$ is in.

$$ \dot{z}=i\omega_1z+a_1z^2\bar{z}+a_2zw\bar{w}+\ldots $$ 
$$ \dot{w}=i\omega_2w+b_1w^2\bar{w}+b_2z\bar{z}w+\ldots $$


$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

$$ \dot{r_1}=\mu_1 r_1+a_1^R r_1^3+a_2^R r_1 r_2^2+\ldots $$ 
$$ \dot{r_2}=\mu_2 r_2+b_1^R r_2^3+b_2^R r_1^2 r_2+\ldots $$ 